好きな用途に利用できるnotebookです。  
データの可視化や簡単なモデルの構築などにご利用下さい。

## 必要なライブラリのimport

In [2]:
import warnings
import time
import sys
import datetime
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

In [3]:
# メモリ削減
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## データの読み込み

In [4]:
# windows
if os.name == 'nt':
    path = '../../../data/processed/'
    data_path = '../../../data/elo-merchant-category-recommendation/'
else:
    if 'KAGGLE_DATA_PROXY_TOKEN' in os.environ.keys():
        path = '/kaggle/input/elo-merchant-category-recommendaton/'

# FIXME:
PATHを修正すること！！！

In [5]:
# FIXME: PATH
train_path = os.path.join(path,'processed20240618_train.csv')
test_path = os.path.join(path,'processed20240618_test.csv')

new_transactions_path = os.path.join(data_path,'new_merchant_transactions.csv')
historical_transactions_path = os.path.join(data_path,'historical_transactions.csv')



In [6]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
new_transactions = pd.read_csv(new_transactions_path,
                               parse_dates=['purchase_date'])
historical_transactions = pd.read_csv(historical_transactions_path,
                                      parse_dates=['purchase_date'])

In [7]:
df_original_train = ['card_id','first_active_month','feature_1','feature_2','feature_3']
# 現時点(20240618,hoji_model)でfeature_importanceの高い値を確認する
df_importance_top10 = ['new_purchase_date_max', 'auth_purchase_date_max',
       'auth_month_diff_mean', 'hist_month_diff_mean',
       'new_purchase_amount_max', 'new_purchase_date_ptp',
       'new_purchase_month_mean', 'auth_purchase_date_ptp',
       'new_month_lag_mean', 'purchase_amount_max_mean']
# 何が役に立ってるんだこれ？
df_train[df_importance_top10].head(20)
# 日付の値めちゃくちゃ大きくなってるけどこれでいいのかなあっていうのと(これミリ秒単位らしい)
# month_diffはcurrent_timeじゃなくて基準日決めたほうがいいよねって思う（やったらほんのちょびっとスコア下がったかも）

,new_purchase_date_max,auth_purchase_date_max,auth_month_diff_mean,hist_month_diff_mean,new_purchase_amount_max,new_purchase_date_ptp,new_purchase_month_mean,auth_purchase_date_ptp,new_month_lag_mean,purchase_amount_max_mean
0,1.525001e+09,1.519551e+09,76.744939,76.846154,-0.29610,4742309.0,3.478261,20977987.0,1.478261,-0.158815
1,1.522393e+09,1.517438e+09,77.731563,77.545455,-0.70170,4887632.0,2.500000,33717687.0,1.500000,0.510664
2,1.524937e+09,1.519759e+09,76.731707,77.000000,-0.70000,0.0,4.000000,35635623.0,2.000000,-0.621090
3,1.524049e+09,1.519818e+09,76.636364,NaN,-0.56700,3625505.0,3.714286,13375339.0,1.714286,-0.097344
4,1.524941e+09,1.519850e+09,76.664062,76.400000,0.45100,4949682.0,3.555556,9405641.0,1.555556,2.259349
5,1.505510e+09,1.501343e+09,83.812500,83.000000,-0.00009,2717302.0,8.750000,17274145.0,1.750000,-0.240106
6,1.523037e+09,1.519402e+09,76.884774,76.882353,-0.68850,2697311.0,3.200000,35783408.0,1.200000,-0.635468
7,1.518986e+09,1.513885e+09,78.473684,78.666667,-0.69730,1085016.0,2.000000,7251596.0,2.000000,-0.347811
8,1.521581e+09,1.512825e+09,77.000000,77.000000,-0.58060,284431.0,3.000000,9751281.0,1.000000,-0.515289
9,1.523196e+09,1.519837e+09,76.848485,76.928571,-0.66600,621055.0,4.000000,35859021.0,2.000000,-0.555582


In [8]:
df_train[df_importance_top10].describe()
# 明日外れ値の処置でもしようかな

,new_purchase_date_max,auth_purchase_date_max,auth_month_diff_mean,hist_month_diff_mean,new_purchase_amount_max,new_purchase_date_ptp,new_purchase_month_mean,auth_purchase_date_ptp,new_month_lag_mean,purchase_amount_max_mean
count,1.799860e+05,2.019170e+05,201917.000000,170958.000000,179986.000000,1.799860e+05,179986.000000,2.019170e+05,179986.000000,201917.000000
mean,1.520773e+09,1.515760e+09,77.891003,77.862506,-0.131695,2.543707e+06,4.084927,2.083236e+07,1.476221,-0.115751
std,6.347417e+06,6.509686e+06,2.393546,2.342374,1.532163,1.743585e+06,2.280113,1.023720e+07,0.330800,1.059925
min,1.488579e+09,1.485935e+09,76.000000,76.000000,-0.747000,0.000000e+00,1.000000,1.520650e+05,1.000000,-0.745349
25%,1.520338e+09,1.514720e+09,76.701613,76.714286,-0.648000,7.465305e+05,3.000000,1.164908e+07,1.230769,-0.568884
50%,1.523342e+09,1.518858e+09,76.807018,77.000000,-0.513000,2.845612e+06,3.500000,1.978835e+07,1.500000,-0.405554
75%,1.524643e+09,1.519662e+09,77.796875,77.888889,-0.181900,4.083677e+06,4.000000,3.094471e+07,1.714286,-0.044238
max,1.525133e+09,1.519862e+09,89.000000,89.000000,76.700000,5.343723e+06,12.000000,3.662692e+07,2.000000,72.958466


In [9]:
df_train[df_importance_top10].isnull().sum()
# なんで欠損してるのか考えたほうがいいなこれ(今度)ー＞多分newにデータはいってないやつがNULLになってるてことだと思うんだよな(newの欠損値がすべて同じのため)(histのNULLもhistにデータ入ってないやつなんじゃないかなあ)


new_purchase_date_max       21931
auth_purchase_date_max          0
auth_month_diff_mean            0
hist_month_diff_mean        30959
new_purchase_amount_max     21931
new_purchase_date_ptp       21931
new_purchase_month_mean     21931
auth_purchase_date_ptp          0
new_month_lag_mean          21931
purchase_amount_max_mean        0
dtype: int64

In [10]:
df_train.describe()

,feature_1,feature_2,feature_3,target,elapsed_time,hist_transactions_count,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_2.0_mean,hist_category_2_3.0_mean,hist_category_2_4.0_mean,hist_category_2_5.0_mean,hist_category_3_A_mean,hist_category_3_B_mean,hist_category_3_C_mean,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_month_std,hist_purchase_date_ptp,hist_purchase_date_min,hist_purchase_date_max,hist_month_lag_mean,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_month_diff_mean,auth_transactions_count,auth_category_1_sum,auth_category_1_mean,auth_category_2_1.0_mean,auth_category_2_2.0_mean,auth_category_2_3.0_mean,auth_category_2_4.0_mean,auth_category_2_5.0_mean,auth_category_3_A_mean,auth_category_3_B_mean,auth_category_3_C_mean,auth_merchant_id_nunique,auth_merchant_category_id_nunique,auth_state_id_nunique,auth_city_id_nunique,auth_subsector_id_nunique,auth_purchase_amount_sum,auth_purchase_amount_mean,auth_purchase_amount_max,auth_purchase_amount_min,auth_purchase_amount_std,auth_installments_sum,auth_installments_mean,auth_installments_max,auth_installments_min,auth_installments_std,auth_purchase_month_mean,auth_purchase_month_max,auth_purchase_month_min,auth_purchase_month_std,auth_purchase_date_ptp,auth_purchase_date_min,auth_purchase_date_max,auth_month_lag_mean,auth_month_lag_max,auth_month_lag_min,auth_month_lag_std,auth_month_diff_mean,new_transactions_count,new_category_1_sum,new_category_1_mean,new_category_2_1.0_mean,new_category_2_2.0_mean,new_category_2_3.0_mean,new_category_2_4.0_mean,new_category_2_5.0_mean,new_category_3_A_mean,new_category_3_B_mean,new_category_3_C_mean,new_merchant_id_nunique,new_merchant_category_id_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_mean,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_month_diff_mean,month_lag_mean,month_lag_std,purchase_amount_count_mean,purchase_amount_count_std,purchase_amount_sum_mean,purchase_amount_sum_std,purchase_amount_mean_mean,purchase_amount_mean_std,purchase_amount_min_mean,purchase_amount_min_std,purchase_amount_max_mean,purchase_amount_max_std,purchase_amount_std_mean,purchase_amount_std_std,installments_count_mean,installments_count_std,installments_sum_mean,installments_sum_std,installments_mean_mean,installments_mean_std,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std,authorized_flag_mean,category_1_purchase_amount_mean,category_1_purchase_amount_min,category_1_purchase_amount_max,category_1_purchase_amount_std,installments_purchase_amount_mean,installments_purchase_amount_min,installments_purchase_amount_max,installments_purchase_amount_std,city_id_purchase_amount_mean,city_id_purchase_amount_min,city_id_purchase_amount_max,city_id_purchase_amount_std,category_1_installments_mean,category_1_installments_min,category_1_installments_max,category_1_installments_std
count,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000,170958.000000,170958.000000,170958.000000,170958.000000,170958.000000,170958.000000,170958.000000,170958.000000,170958.000000,170958.000000,170958.00

In [15]:
df_train['rate_new/hist_purchase_date_max'] = df_train['new_purchase_date_max']/df_train['hist_purchase_date_max']

0         1.514385e+09
1         1.514467e+09
2         1.492801e+09
3                  NaN
4         1.519759e+09
              ...     
201912    1.518114e+09
201913    1.511473e+09
201914    1.513959e+09
201915    1.509124e+09
201916    1.518705e+09
Name: hist_purchase_date_max, Length: 201917, dtype: float64